In [1]:
!pip -q install segmentation-models-pytorch
!pip -q install albumentations --upgrade
!pip -q install opencv-python --upgrade

     |████████████████████████████████| 88 kB 2.3 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 376 kB 23.9 MB/s 
     |████████████████████████████████| 113 kB 4.2 MB/s 
     |████████████████████████████████| 48.3 MB 2.0 MB/s 
     |████████████████████████████████| 60.9 MB 64 kB/s 


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATASET =     {#'mask':'uwmgi-mask-dataset.zip',
               #'masks':'masks.zip',
               'train': 'uw-madison-gi-tract-image-segmentation.zip'}
METADATA=     {#'train_csv':'train.csv',
               #'train_processed':'train_processed.csv',
               #'sample_sub':'sample_submission.csv',
               'config':'config.yaml'}

INPUT_DIR ='/content/drive/MyDrive/kaggle/gi_t_is/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/gi_t_is/output/'

WORK_DIR='/content/'
EXPERIMENT='lum_unetb3_320_384'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'

DATASET_DIR = INPUT_DIR
METADATA_DIR = INPUT_DIR

IMPORT_DIR = '/content/drive/MyDrive/kaggle/gi_t_is/nbs/py/'
LUMINIDE_PY=IMPORT_DIR+'luminide_code/'

In [5]:
def copy_dataset(ds_dict, ds_dir, work_dir):
  for record in ds_dict:
    print('copy', ds_dir+ds_dict[record], ' to', work_dir)
    !cp {ds_dir+ds_dict[record]} {work_dir}
    print ('unzip -q ',work_dir+ds_dict[record],' -d ',work_dir)
    !unzip -q  {work_dir+ds_dict[record]} -d {work_dir}
    print ('rm ',work_dir+ds_dict[record])
    !rm {work_dir+ds_dict[record]}
def copy_metadata(md_dict,md_dir,work_dir):
  for record in md_dict:
    print('copy ', md_dir+md_dict[record],' to ',work_dir)
    !cp {md_dir+md_dict[record]} {work_dir}

copy_dataset(DATASET,DATASET_DIR, WORK_DIR)
copy_metadata(METADATA,METADATA_DIR,WORK_DIR)
!ls /content/

copy /content/drive/MyDrive/kaggle/gi_t_is/input/uw-madison-gi-tract-image-segmentation.zip  to /content/
unzip -q  /content/uw-madison-gi-tract-image-segmentation.zip  -d  /content/
rm  /content/uw-madison-gi-tract-image-segmentation.zip
copy  /content/drive/MyDrive/kaggle/gi_t_is/input/config.yaml  to  /content/
config.yaml  drive  sample_data  sample_submission.csv	train  train.csv


In [6]:
import os
import sys
sys.path.append(LUMINIDE_PY)
from glob import glob
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.utils.data as data
import gc
import cupy as cp

import multiprocessing as mp
import segmentation_models_pytorch as smp
import cv2
from tqdm import tqdm
tqdm.pandas()
import albumentations as A
from util import get_class_names, make_test_augmenter, get_id
from inference import create_model,get_img_shape,pad_mask,resize_mask,rle_encode

Running on cuda


In [7]:
!ls train

case101  case116  case130  case142  case156  case32  case47  case67  case89
case102  case117  case131  case143  case16   case33  case49  case7   case9
case107  case118  case133  case144  case18   case34  case53  case74  case90
case108  case119  case134  case145  case19   case35  case54  case77  case91
case11	 case121  case135  case146  case2    case36  case55  case78  case92
case110  case122  case136  case147  case20   case40  case58  case80
case111  case123  case138  case148  case22   case41  case6   case81
case113  case124  case139  case149  case24   case42  case63  case84
case114  case125  case140  case15   case29   case43  case65  case85
case115  case129  case141  case154  case30   case44  case66  case88


In [9]:
CKPT_DIR = OUTPUT_DIR +  'unet_effnetb0_2.5d_320x384_ckpt_ds'
#CKPT_DIRS = {'fold0':'/kaggle/input/ver13-3-efb2-320x384x1-bs128-f0-vd-09146',
#             'fold123': '/kaggle/input/ver13-3-efb2-320x384x1-bs128-f-1-2-3',
#             'fold4': '/kaggle/input/ver13-3-efb2-320x384x1-bs128-f4-vd09002'}

In [10]:
class CFG:
    seed          = 101
    debug         = False # set debug=False for Full Training
    exp_name      = 'v4'
    comment       = 'unet-efficientnet_b0-320x384'
    model_name    = 'Unet'
    backbone      = 'efficientnet-b0'
    backbone2     = 'efficientnet-b2'
    train_bs      = 64
    valid_bs      = train_bs*2
    img_size      = [320, 384]
    epochs        = 16
    lr            = 2e-3
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = max(1, 32//train_bs)
    n_fold        = 5
    folds         = [0]
    num_classes   = 3
    input_chnls   = 1
    thr           = 0.4
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
def build_model():
    model = smp.Unet(
        encoder_name=CFG.backbone,      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=CFG.num_classes,        # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to(CFG.device)
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

def build_model2():
    model = smp.Unet(
        encoder_name=CFG.backbone2,      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=CFG.input_chnls,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=CFG.num_classes,        # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to(CFG.device)
    return model

In [12]:
model_paths  = glob(f'{CKPT_DIR}/best_epoch*.bin')
#model_paths2 = []
#for f in CKPT_DIRS:
#    model_paths2+=glob(f'{CKPT_DIRS[f]}/best_valid*.bin')

In [13]:
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
#     row['id'] = f'case{case}_day{day}_slice_{slice_}'
    return row

In [14]:
!ls train/case101/case101_day26/scans/*slice_0095*

train/case101/case101_day26/scans/slice_0095_266_266_1.50_1.50.png


In [15]:
test_id = 'case101_day26_slice_0095'
test_msk_25 = np.zeros((266,266,3))
test_msk = np.zeros((266,266,3))

In [16]:
sub_df = pd.read_csv('/content/sample_submission.csv')
if not len(sub_df):
    debug = True
    sub_df = pd.read_csv('/content/train.csv')
    sub_df = sub_df.sort_values('id')[:384*3]
    sub_df = sub_df.drop(columns=['class','segmentation']).drop_duplicates()
else:
    debug = False
    sub_df = sub_df.sort_values('id')
    sub_df = sub_df.drop(columns=['class','predicted']).drop_duplicates()
sub_df = sub_df.progress_apply(get_metadata,axis=1)

100%|██████████| 384/384 [00:00<00:00, 639.13it/s]


In [16]:
#sub_df=sub_df[sub_df.case==133]

In [17]:
if debug:
    paths = glob(f'/content/train/**/*png',recursive=True)     
    paths = sorted(paths)[:384]
else:
    paths = glob(f'/content/test/**/*png',recursive=True)
    paths = sorted(paths)
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.progress_apply(path2info, axis=1)
path_df.head()

100%|██████████| 384/384 [00:01<00:00, 332.51it/s]


,image_path,height,width,case,day,slice
0,/content/train/case101/case101_day20/scans/sli...,266,266,101,20,1
1,/content/train/case101/case101_day20/scans/sli...,266,266,101,20,2
2,/content/train/case101/case101_day20/scans/sli...,266,266,101,20,3
3,/content/train/case101/case101_day20/scans/sli...,266,266,101,20,4
4,/content/train/case101/case101_day20/scans/sli...,266,266,101,20,5


In [ ]:
#path_df=path_df[path_df.case==133]
#path_df.head()

In [18]:
test_df_25 = sub_df.merge(path_df, on=['case','day','slice'], how='left')
test_df_25.head()

,id,case,day,slice,image_path,height,width
0,case101_day20_slice_0001,101,20,1,/content/train/case101/case101_day20/scans/sli...,266,266
1,case101_day20_slice_0002,101,20,2,/content/train/case101/case101_day20/scans/sli...,266,266
2,case101_day20_slice_0003,101,20,3,/content/train/case101/case101_day20/scans/sli...,266,266
3,case101_day20_slice_0004,101,20,4,/content/train/case101/case101_day20/scans/sli...,266,266
4,case101_day20_slice_0005,101,20,5,/content/train/case101/case101_day20/scans/sli...,266,266


In [19]:
channels=3
stride=2
for i in range(channels):
    test_df_25[f'image_path_{i:02}'] = test_df_25.groupby(['case','day'])['image_path'].shift(-i*stride).fillna(method="ffill")
test_df_25['image_paths'] = test_df_25[[f'image_path_{i:02d}' for i in range(channels)]].values.tolist()
test_df_25.image_paths[0]

['/content/train/case101/case101_day20/scans/slice_0001_266_266_1.50_1.50.png',
 '/content/train/case101/case101_day20/scans/slice_0003_266_266_1.50_1.50.png',
 '/content/train/case101/case101_day20/scans/slice_0005_266_266_1.50_1.50.png']

In [20]:
def load_img_25(path, size=CFG.img_size):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    shape0 = np.array(img.shape[:2])
    resize = np.array(size)
    if np.any(shape0!=resize):
        diff = resize - shape0
        pad0 = diff[0]
        pad1 = diff[1]
        pady = [pad0//2, pad0//2 + pad0%2]
        padx = [pad1//2, pad1//2 + pad1%2]
        img = np.pad(img, [pady, padx])
        img = img.reshape((*resize))
    return img, shape0

def load_imgs_25(img_paths, size=CFG.img_size):
    #print(len(img_paths))
    #print(img_paths)
    imgs = np.zeros((*size, len(img_paths)), dtype=np.float32)
    for i, img_path in enumerate(img_paths):
        if i==0:
            img, shape0 = load_img_25(img_path, size=size)
        else:
            img, _ = load_img_25(img_path, size=size)
        img = img.astype('float32') # original is uint16
        mx = np.max(img)
        if mx:
            img/=mx # scale image to [0, 1]
        imgs[..., i]+=img
    return imgs, shape0

In [21]:
class VisionDataset(data.Dataset):
    def __init__(
            self, df,df_25, conf, input_dir, imgs_dir,
            class_names, transform, is_test=False, subset=100):
        self.conf = conf
        self.transform = transform
        self.is_test = is_test
        

        if subset != 100:
            assert subset < 100
            # train and validate on subsets
            num_rows = df.shape[0]*subset//100
            df = df.iloc[:num_rows]
            

        files = df['img_files']
        self.files = [os.path.join(input_dir, imgs_dir, f) for f in files]
        self.masks = [os.path.join(input_dir, 'masks', f) for f in files]

        self.df_25         = df_25
        #self.img_paths_25  = df_25['image_paths'].tolist()
        #self.img_path_25 =   df_25['image_path'].tolist()

    def resize(self, img, interp):
        return  cv2.resize(
            img, (self.conf.w_image_size,self.conf.h_image_size), interpolation=interp)

    def load_slice(self, img_file, diff):
        slice_num = os.path.basename(img_file).split('_')[1]
        filename = (
            img_file.replace(
                'slice_' + slice_num,
                'slice_' + str(int(slice_num) + diff).zfill(4)))
        
        if os.path.exists(filename):
            return cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        return None
    def __getitem__(self, index):
        conf = self.conf
        img_file = self.files[index]
        rec = self.df_25[self.df_25.image_path==img_file]
        assert len(rec)==1, f'record not found! {img_file}'
        # read 5 slices into one image
        imgs = [self.load_slice(img_file, i) for i in range(-2, 3)]
        
        if imgs[3] is None:
            imgs[3] = imgs[2]
        if imgs[4] is None:
            imgs[4] = imgs[3]
        if imgs[1] is None:
            imgs[1] = imgs[2]
        if imgs[0] is None:
            imgs[0] = imgs[1]
        img = np.stack(imgs, axis=2)

        img = img.astype(np.float32)
        max_val = img.max()
        if max_val != 0:
            img /= max_val
        img = self.resize(img, cv2.INTER_AREA)
        
        img_path_25  = rec.reset_index().image_paths[0]
        img_25 = []
        img_25, shape0_25 = load_imgs_25(img_path_25)
        img_25 = np.transpose(img_25, (2, 0, 1))

        if self.is_test:
            msk = 0
            result = self.transform(image=img)
            img = result['image']
        else:
            # read mask
            msk_file = self.masks[index]
            msk = cv2.imread(msk_file, cv2.IMREAD_UNCHANGED)
            msk = self.resize(msk, cv2.INTER_NEAREST)
            msk = msk.astype(np.float32)
            result = self.transform(image=img, mask=msk)
            img, msk = result['image'], result['mask']
        return img, msk,torch.tensor(img_25)

    def __len__(self):
        return len(self.files)

In [28]:
'/content/train/case133/case133_day0/scans/slice_0080_360_310_1.50_1.50.png'.split('/')[3]

'case133'

In [22]:
def create_test_loader(conf, input_dir, class_names, img_dir='test', subset=100):
    test_aug = make_test_augmenter(conf)
    test_df  = pd.DataFrame()   
    img_files = []
    img_dir = img_dir
    subdir = ''
    while len(img_files) == 0 and len(subdir) < 10:
        img_files = sorted(glob(f'{input_dir}/{img_dir}/{subdir}*.png'))
        subdir += '*/'
        if len(subdir) > 10:
            return None
    # delete common prefix from paths
    img_files = [f.replace(f'{input_dir}/{img_dir}/', '') for f in img_files] #if f.split('/')[3]=='case133']

    test_df['img_files'] = img_files
    
    test_dataset = VisionDataset(
        test_df, test_df_25,conf, input_dir, img_dir,
        class_names, test_aug, is_test=True, subset=subset)
    print(f'{len(test_dataset)} examples in test set')
    loader = data.DataLoader(
        test_dataset, batch_size=conf.batch_size, shuffle=False,
        num_workers=mp.cpu_count(), 
        pin_memory=False)
    return loader, test_df

In [23]:
check_sum = 0
check_sum_25 = 0

In [24]:
def inference(input_dir, model_dir, thresh):
    global test_msk
    global check_sum
    meta_file = os.path.join(input_dir, 'train.csv')
    train_df = pd.read_csv(meta_file, dtype=str)
    class_names = np.array(get_class_names(train_df))
    num_classes = len(class_names)

    model, conf = create_model(model_dir, num_classes)
    if debug:
        loader, df = create_test_loader(conf, input_dir, class_names, img_dir='train', subset=1)
        subm = pd.read_csv(f'{input_dir}/train.csv')
        del subm['segmentation']
    else:
        loader, df = create_test_loader(conf, input_dir, class_names)
        subm = pd.read_csv(f'{input_dir}/sample_submission.csv')
        del subm['predicted']
        
    img_files = df['img_files']

    ids = []
    classes = []
    masks = []
    img_idx = 0
    sigmoid = nn.Sigmoid()
    model.eval()
    with torch.no_grad():
        for images, _, images_25 in tqdm(loader):
            images = images.to(device)
            outputs = model(images)
            preds = sigmoid(outputs)
            size = preds.size()
            preds_25 = torch.zeros((size[0], 3, size[3], size[2]), device=CFG.device, dtype=torch.float32)
            
            for path in model_paths:
                model_25d = load_model(path)
                images_25 = images_25.to(CFG.device, dtype=torch.float)
                out   = model_25d(images_25) # .squeeze(0) # removing batch axis
                out   = nn.Sigmoid()(out) # removing channel axis
                #out = out.permute((0,1,3,2))
                preds_25+=out
            
            preds_25 /= 5.0
            preds_25 = preds_25.cpu().numpy()
            
            
            preds = preds.cpu().numpy()
                       
            #preds[preds >= thresh] = 1
            #preds[preds < thresh] = 0
                  
            for pred,pred_25 in zip(preds,preds_25):
                img_file = img_files[img_idx]
                img_idx += 1
                img_id = get_id(img_file)
                height, width = get_img_shape(img_file)
                
                for class_id, class_name in enumerate(class_names):
                    #print(f'class_id:{class_id},class_name: {class_name}')
                    mask = pred[class_id]
                    mask_25 = pred_25[class_id]
                    if conf.crop_size < 1.0:
                        mask = pad_mask(conf, mask)
                    
                    mask = resize_mask(mask, height, width)
                    
                    shape0 = np.array([height, width])
                    resize = np.array([320, 384])
                    if np.any(shape0!=resize):
                        diff = resize - shape0
                        pad0 = diff[0]
                        pad1 = diff[1]
                        pady = [pad0//2, pad0//2 + pad0%2]
                        padx = [pad1//2, pad1//2 + pad1%2]
                        mask_25 = mask_25[pady[0]:-pady[1], padx[0]:-padx[1]]
                        #mask_25 = np.transpose(mask_25)
                       
                    #mask_total = (mask+mask_25)/2.0
                    mask_25[mask_25 > thresh] = 1
                    mask_25[mask_25 <= thresh] = 0
                    check_sum += mask_25.sum()
                    if img_id == test_id:
                      test_msk[:,:,class_id] = mask_25
                    enc_mask = '' if mask_25.sum() == 0 else rle_encode(mask_25)
                    ids.append(img_id)
                    classes.append(class_name)
                    masks.append(enc_mask)

    pred_df = pd.DataFrame({'id': ids, 'class': classes, 'predicted': masks})
    
    if pred_df.shape[0] > 0:
        # sort according to the given order and save to a csv file
        subm = subm.merge(pred_df, on=['id', 'class'])
    subm.to_csv('submission.csv', index=False)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Running on {device}')

Running on cuda


In [26]:
input_dir = '/content'
model_dir = OUTPUT_DIR+'LUMINIDE_TEST_PIPELINE'
thr = 0.4

In [27]:
inference(input_dir,model_dir,thr)

384 examples in test set


100%|██████████| 12/12 [00:27<00:00,  2.30s/it]


In [28]:
class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=False, transforms=None):
        self.df         = df
        self.label      = label
        self.img_paths  = df['image_paths'].tolist()
        self.ids        = df['id'].tolist()
        if 'msk_path' in df.columns:
            self.msk_paths  = df['mask_path'].tolist()
        else:
            self.msk_paths = None
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path  = self.img_paths[index]
        id_       = self.ids[index]
        img = []
        img, shape0 = load_imgs_25(img_path)
        h, w = shape0
        if self.label:
            msk_path = self.msk_paths[index]
            msk = load_msk(msk_path)
            if self.transforms:
                data = self.transforms(image=img, mask=msk)
                img  = data['image']
                msk  = data['mask']
            img = np.transpose(img, (2, 0, 1))
            msk = np.transpose(msk, (2, 0, 1))
            return torch.tensor(img), torch.tensor(msk)
        else:
            if self.transforms:
                data = self.transforms(image=img)
                img  = data['image']
            img = np.transpose(img, (2, 0, 1))
            return torch.tensor(img), id_, h, w

In [29]:
data_transforms = {
    "train": A.Compose([
#         A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
#         A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=5, p=0.5),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
# #             A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        ], p=0.25),
#         A.CoarseDropout(max_holes=8, max_height=CFG.img_size[0]//20, max_width=CFG.img_size[1]//20,
#                          min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        ], p=1.0),
    
    "valid": A.Compose([
#         A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        ], p=1.0)
}

In [30]:
def mask2rle(msk, thr=0.5):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    msk    = cp.array(msk)
    pixels = msk.flatten()
    pad    = cp.array([0])
    pixels = cp.concatenate([pad, pixels, pad])
    runs   = cp.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def masks2rles(msks, ids, heights, widths):
    pred_strings = []; pred_ids = []; pred_classes = [];
    global test_msk_25
    global check_sum_25
    for idx in range(msks.shape[0]):
        msk = msks[idx]
        height = heights[idx].item()
        width = widths[idx].item()
        shape0 = np.array([height, width])
        resize = np.array([320, 384])
        if ids[idx] == test_id:
          print('')
          print('shape0: ', shape0)
          print('resize: ', resize)
          print('')

        if np.any(shape0!=resize):
            diff = resize - shape0
            pad0 = diff[0]
            pad1 = diff[1]
            pady = [pad0//2, pad0//2 + pad0%2]
            padx = [pad1//2, pad1//2 + pad1%2]
            msk = msk[pady[0]:-pady[1], padx[0]:-padx[1], :]
            msk = msk.reshape((*shape0, 3))
        
        check_sum_25 += msk.sum()
        if ids[idx] == test_id:
          test_msk_25 = msk

        rle = [None]*3
        for midx in [0, 1, 2]:
            rle[midx] = mask2rle(msk[...,midx])
        pred_strings.extend(rle)
        pred_ids.extend([ids[idx]]*len(rle))
        pred_classes.extend(['large_bowel', 'small_bowel', 'stomach'])
    return pred_strings, pred_ids, pred_classes

In [31]:
@torch.no_grad()
def infer(model_paths, test_loader, num_log=1, thr=CFG.thr):
    pred_strings = []; pred_ids = []; pred_classes = [];
    for idx, (img, ids, heights, widths) in enumerate(tqdm(test_loader, total=len(test_loader), desc='Infer ')):
        img = img.to(CFG.device, dtype=torch.float) # .squeeze(0)
        size = img.size()
        msk = []
        msk = torch.zeros((size[0], 3, size[2], size[3]), device=CFG.device, dtype=torch.float32)
        for path in model_paths:
            model = load_model(path)
            out   = model(img) # .squeeze(0) # removing batch axis
            out   = nn.Sigmoid()(out) # removing channel axis
            msk+=out/len(model_paths)
        msk = (msk.permute((0,2,3,1))>thr).to(torch.uint8).cpu().detach().numpy() # shape: (n, h, w, c)
        result = masks2rles(msk, ids, heights, widths)
        pred_strings.extend(result[0])
        pred_ids.extend(result[1])
        pred_classes.extend(result[2])
        gc.collect()
        torch.cuda.empty_cache()
    return pred_strings, pred_ids, pred_classes

In [32]:
test_dataset = BuildDataset(test_df_25, transforms=data_transforms['valid'])
test_loader  = data.DataLoader(test_dataset, batch_size=CFG.valid_bs, 
                          num_workers=mp.cpu_count(), shuffle=False, pin_memory=False)

In [33]:
pred_strings, pred_ids, pred_classes = infer(model_paths, test_loader)

Infer : 100%|██████████| 3/3 [00:16<00:00,  5.09s/it]


shape0:  [266 266]
resize:  [320 384]



Infer : 100%|██████████| 3/3 [00:16<00:00,  5.57s/it]


In [34]:
check_sum == check_sum_25

True

In [35]:
check_sum_25

437122.0

In [36]:
check_sum

437122.0

In [37]:
test_msk[:,:,2].sum()

450.0

In [38]:
test_msk_25[:,:,2].sum()

450

In [39]:
np.array_equal(test_msk,test_msk_25)

True

In [40]:
pred_df = pd.DataFrame({
    "id":pred_ids,
    "class":pred_classes,
    "predicted":pred_strings
})
if not debug:
    sub_df = pd.read_csv('/content/sample_submission.csv')
    del sub_df['predicted']
else:
    sub_df = pd.read_csv('/content/train.csv')
    #sub_df = sub_df.sort_values('id')#[:384*3]
    del sub_df['segmentation']
    
sub_df = sub_df.merge(pred_df, on=['id','class'])
sub_df.to_csv('submission_25.csv',index=False)
display(sub_df.head(5))

,id,class,predicted
0,case101_day20_slice_0001,large_bowel,
1,case101_day20_slice_0001,small_bowel,
2,case101_day20_slice_0001,stomach,
3,case101_day20_slice_0002,large_bowel,
4,case101_day20_slice_0002,small_bowel,


In [41]:
subm = pd.read_csv('/content/submission.csv')

In [42]:
subm_25 = pd.read_csv('/content/submission_25.csv')

In [43]:
subm_25[subm_25['id']==	test_id]

,id,class,predicted
1146,case101_day26_slice_0095,large_bowel,19327 10 19584 26 19845 36 20108 42 20373 46 2...
1147,case101_day26_slice_0095,small_bowel,23854 4 24116 9 24380 17 24645 23 24910 26 251...
1148,case101_day26_slice_0095,stomach,20609 6 20871 15 21135 19 21400 22 21665 25 21...


In [44]:
subm[subm['id']== test_id]

,id,class,predicted
1146,case101_day26_slice_0095,large_bowel,19327 10 19584 26 19845 36 20108 42 20373 46 2...
1147,case101_day26_slice_0095,small_bowel,23854 4 24116 9 24380 17 24645 23 24910 26 251...
1148,case101_day26_slice_0095,stomach,20609 6 20871 15 21135 19 21400 22 21665 25 21...
